# HOMEWORK 5: TEXT CLASSIFICATION
In this homework, you will create models to classify texts from TRUE call-center. There are two classification tasks:
1. Action Classification: Identify which action the customer would like to take (e.g. enquire, report, cancle)
2. Object Classification: Identify which object the customer is referring to (e.g. payment, truemoney, internet, roaming) 

In this homework, you are asked to do the following tasks:
1. Data Cleaning
2. Preprocessing data for keras
3. Build and evaluate a model for "action" classification
4. Build and evaluate a model for "object" classification
5. Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go 


Note: we have removed phone numbers from the dataset for privacy purposes. 

## Import Libs

In [1]:
%matplotlib inline
import pandas as pd
import sklearn
import numpy as np
from IPython.display import display
import os
import matplotlib.pyplot as plt
import pickle
import collections

In [2]:
from keras.models import Model
from keras.layers import Input, Dense, Embedding, Conv1D, Dropout, GRU, Bidirectional, Conv2D, LSTM
from keras.layers import Reshape, Activation, Flatten, TimeDistributed,MaxPooling1D, MaxPooling2D
from keras.preprocessing import sequence
from keras.layers.merge import Dot
from keras.optimizers import Adam
from keras.callbacks import TensorBoard, ModelCheckpoint, ReduceLROnPlateau


/home/ekapolc/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Loading data
First, we load the data from disk into a Dataframe.

A Dataframe is essentially a table, or 2D-array/Matrix with a name for each column.

In [5]:
data_df = pd.read_csv('clean-phone-data-for-students.csv')

Let's preview the data.

In [6]:
# Show the top 5 rows
display(data_df.head())
# Summarize the data
data_df.describe()

,Sentence Utterance,Action,Object
0,<PHONE_NUMBER_REMOVED> ผมไปจ่ายเงินที่ Counte...,enquire,payment
1,internet ยังความเร็วอยุ่เท่าไหร ครับ,enquire,package
2,ตะกี้ไปชำระค่าบริการไปแล้ว แต่ยังใช้งานไม่ได้...,report,suspend
3,พี่ค่ะยังใช้ internet ไม่ได้เลยค่ะ เป็นเครื่อ...,enquire,internet
4,ฮาโหล คะ พอดีว่าเมื่อวานเปิดซิมทรูมูฟ แต่มันโ...,report,phone_issues


,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


## Data cleaning

We call the DataFrame.describe() again.
Notice that there are 33 unique labels/classes for object and 10 unique labels for action that the model will try to predict.
But there are unwanted duplications e.g. Idd,idd,lotalty_card,Lotalty_card

Also note that, there are 13389 unqiue sentence utterances from 16175 utterances. You have to clean that too!

## #TODO 1: 
You will have to remove unwanted label duplications as well as duplications in text inputs. 
Also, you will have to trim out unwanted whitespaces from the text inputs. 
This shouldn't be too hard, as you have already seen it in the demo.



In [7]:
display(data_df.describe())
display(data_df.Object.unique())
display(data_df.Action.unique())

,Sentence Utterance,Action,Object
count,16175,16175,16175
unique,13389,10,33
top,บริการอื่นๆ,enquire,service
freq,97,10377,2525


array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nonTrueMove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd',
       'TrueMoney', 'garbage', 'Payment', 'IDD', 'ringtone', 'Idd',
       'rate', 'loyalty_card', 'contact', 'officer', 'Balance', 'Service',
       'Loyalty_card'], dtype=object)

array(['enquire', 'report', 'cancel', 'Enquire', 'buy', 'activate',
       'request', 'Report', 'garbage', 'change'], dtype=object)

In [8]:
# TODO1: Data cleaning
data_df['Action']=data_df['Action'].str.lower().copy()
data_df['Object']=data_df['Object'].str.lower().copy()
data_df = data_df.rename(index=str, columns={"Sentence Utterance": "Sent", "Action": "Action", "Object":"Object"})
display(data_df.describe())
display(data_df.Action.unique())
display(data_df.Object.unique())

,Sent,Action,Object
count,16175,16175,16175
unique,13389,8,26
top,บริการอื่นๆ,enquire,service
freq,97,10484,2528


array(['enquire', 'report', 'cancel', 'buy', 'activate', 'request',
       'garbage', 'change'], dtype=object)

array(['payment', 'package', 'suspend', 'internet', 'phone_issues',
       'service', 'nontruemove', 'balance', 'detail', 'bill', 'credit',
       'promotion', 'mobile_setting', 'iservice', 'roaming', 'truemoney',
       'information', 'lost_stolen', 'balance_minutes', 'idd', 'garbage',
       'ringtone', 'rate', 'loyalty_card', 'contact', 'officer'],
      dtype=object)

In [9]:
import re
#data_df['Sent'] = data_df['Sent'].apply(lambda k : re.sub(r'["|–|\'|:|;|?|$|!|~|\n|\t|-|#|+|<|>|/|\\|\|{|}|\[|\]|`|0|1|2|3|4|5|6|7|8|9|*|.|%|@|$|^|&|=|:|(|)|-|_]', r'', k))
data_df['Sent'] = data_df['Sent'].apply(lambda k : re.sub(r'["|–|\'|:|;|?|$|!|~|\n|\t|-|#|+|<|>|/|\\|\|{|}|\[|\]|`|*|%|@|$|^|&|=|:|(|)|-|_]', r'', k))

In [10]:
data_df = data_df.drop_duplicates("Sent", keep="first")
display(data_df.describe())

,Sent,Action,Object
count,13380,13380,13380
unique,13380,8,26
top,ผมยากทราบว่า ผมใช้ package internet อะไรอยุ่ครับ,enquire,service
freq,1,8650,2111


## #TODO 2: Preprocessing data for Keras
You will be using Keras in this assignment. Please show us how you prepare your data for keras.
Don't forget to split data into train and test sets (+ validation set if you want)

In [11]:
# TODO2: Preprocessing data for Keras
data_set = data_df.copy()
data_set['Sent'] = data_df['Sent'].apply(lambda row: list(row))

In [12]:
# Create a character map
CHARS = [
  '\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+',
  ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8',
  '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E',
  'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R',
  'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '^', '_',
  'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm',
  'n', 'o', 'other', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y',
  'z', '}', '~', 'ก', 'ข', 'ฃ', 'ค', 'ฅ', 'ฆ', 'ง', 'จ', 'ฉ', 'ช',
  'ซ', 'ฌ', 'ญ', 'ฎ', 'ฏ', 'ฐ', 'ฑ', 'ฒ', 'ณ', 'ด', 'ต', 'ถ', 'ท',
  'ธ', 'น', 'บ', 'ป', 'ผ', 'ฝ', 'พ', 'ฟ', 'ภ', 'ม', 'ย', 'ร', 'ฤ',
  'ล', 'ว', 'ศ', 'ษ', 'ส', 'ห', 'ฬ', 'อ', 'ฮ', 'ฯ', 'ะ', 'ั', 'า',
  'ำ', 'ิ', 'ี', 'ึ', 'ื', 'ุ', 'ู', 'ฺ', 'เ', 'แ', 'โ', 'ใ', 'ไ',
  'ๅ', 'ๆ', '็', '่', '้', '๊', '๋', '์', 'ํ', '๐', '๑', '๒', '๓',
  '๔', '๕', '๖', '๗', '๘', '๙', '‘', '’', '\ufeff'
]
CHARS_MAP = {v: k for k, v in enumerate(CHARS)}

In [13]:
def create_n_gram_df(df, n_pad):
    """
    Given an input dataframe, create a feature dataframe of shifted characters
    Input:
    df: timeseries of size (N)
    n_pad: the number of context. For a given character at position [idx],
    character at position [idx-n_pad/2 : idx+n_pad/2] will be used 
    as features for that character.

    Output:
    dataframe of size (N * n_pad) which each row contains the character, 
    n_pad_2 characters to the left, and n_pad_2 characters to the right
    of that character.
    """
    n_pad_2 = int((n_pad - 1)/2)
    for i in range(n_pad_2):
        df['char-{}'.format(i+1)] = df['char'].shift(i + 1)
        df['char{}'.format(i+1)] = df['char'].shift(-i - 1)
    return df[n_pad_2: -n_pad_2]

In [27]:
def prepare_feature(input_string):
    """
    Transform the path to a directory containing processed files 
    into a feature matrix and output array
    Input:
    best_processed_path: str, path to a processed version of the BEST dataset
    option: str, 'train' or 'test'
    """
    # we use padding equals 21 here to consider 10 characters to the left
    # and 10 characters to the right as features for the character in the middle
    n_pad = 21
    n_pad_2 = int((n_pad - 1)/2)
    pad = [{'char' : ' '}]
    df_pad = pd.DataFrame(pad * n_pad_2)

#     df = pd.DataFrame(data=best_processed_path['sent'][0], columns=['char'])
    df = pd.DataFrame(data=input_string, columns=['char'])
    # pad with empty string feature
    df = pd.concat((df_pad, df, df_pad))
    
    # map characters to numbers, use 'other' if not in the predefined character set.
    df['char'] = df['char'].map(lambda x: CHARS_MAP.get(x, 80))
    # Use nearby characters as features
    df_with_context = create_n_gram_df(df, n_pad=n_pad)

    char_row = ['char' + str(i + 1) for i in range(n_pad_2)] + \
             ['char-' + str(i + 1) for i in range(n_pad_2)] + ['char']

    # convert pandas dataframe to numpy array to feed to the model
    x_char = df_with_context[char_row].as_matrix()

    return x_char

In [28]:
# Tokenize model
def get_my_tokenize_model():
    input1 = Input(shape=(21,))
    x = Embedding(178,8)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    out = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='binary_crossentropy',
                 metrics=['acc'])          
    return model

In [29]:
## LOAD Tokenize model
weight_path_model_best='/data/model_best.h5'

tokenize_model = get_my_tokenize_model()
tokenize_model.load_weights(weight_path_model_best)

In [32]:
def map_pred_to_word(y_pred,sent):
    out = []
    w = ''
    for i in range(len(y_pred)):
        if(y_pred[i] == 1):
            out.append(w)
            w = sent[i]
        else:
            w += sent[i]
#         t = w.strip()
#         if(t != ''):
    out.append(w.strip())
    return out[1:]

In [68]:
%%time
from tqdm import tqdm

tokenized_sent = []
for i in tqdm(range(len(data_set))):
    chars_array = prepare_feature(data_set['Sent'][i])
    y_pred = tokenize_model.predict(chars_array)
    #map probability to class
    prob_to_class = lambda p: 1 if p[0]>=0.5 else 0
    y_pred = np.apply_along_axis(prob_to_class,1,y_pred)
    tokenized_sent.append(map_pred_to_word(y_pred,data_df['Sent'][i]))

100%|██████████| 13380/13380 [04:11<00:00, 53.29it/s]

CPU times: user 4min 33s, sys: 9.29 s, total: 4min 43s
Wall time: 4min 11s


In [69]:
prepared_data = pd.DataFrame(data={'Sent':np.array(tokenized_sent), 'Action':data_df['Action'].as_matrix(),\
                                  'Object':data_df['Object'].as_matrix()} , columns=['Sent','Action','Object'])

In [70]:
# save tokenize
with open('/data/tokenized-truevoice-clean', 'wb') as f:
    pickle.dump(prepared_data, f)

In [14]:
# load tokenize
with open('/data/tokenized-truevoice-clean', 'rb') as f:
    tokenized_sent = pickle.load(f)

In [15]:
## read fasttext
ftext_w = {}
with open('/data/fasttext/wiki.th.vec', 'r') as f:
    embeded_w = f.readlines()
for line in embeded_w:
    values = line.rstrip().rsplit(' ')
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    ftext_w[word] = coefs

In [16]:
max_len = 55
def create_index(input_text):
    count_word = 0
    words = []
    
    for sent in input_text:
        for w in sent:
            words.append(w.strip('\n'))
            count_word +=1
    
    word_count = list()
    #use set and len to get the number of unique words
    word_count.extend(collections.Counter(words).most_common(len(set(words))))
    
    #include a token for unknown word
    threshold = 2
    num_UNK = 0
    index = len(word_count) - 1
    rare_word = set()
    
    
    while(word_count[index][1] <= threshold):
        num_UNK += word_count[index][1]
        rare_word.add(word_count[index][0])
        index -= 1
    
    word_count = word_count[:index+1]
    word_count.append(("UNK",num_UNK))
    word_count = sorted(word_count, key=lambda x: -x[1])
    
    print(num_UNK , num_UNK/count_word)       

    #print out 10 most frequent words
    
    print(word_count[:10])
    dictionary = dict()
    dictionary["for_keras_zero_padding"] = 0
    
    for word in word_count:
        dictionary[word[0]] = len(dictionary)
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    
    dataset = list()
    for sent in input_text:
        dataset.append([])
        for word in sent[:max_len]:
            if(word not in rare_word):
                dataset[-1].append(dictionary[word])
            else:
                dataset[-1].append(dictionary["UNK"])

    return dataset, dictionary, reverse_dictionary

dataset ,dictionary,reverse_dictionary = create_index(tokenized_sent['Sent'])

5600 0.028908447977699196
[(' ', 24314), ('ค่ะ', 5957), ('UNK', 5600), ('จะ', 5448), ('ครับ', 4628), ('ได้', 3885), ('ไม่', 3656), ('ใช้', 3388), ('ว่า', 2772), ('ผม', 2689)]


In [17]:
len(dataset)

13380

In [34]:
action_mapped = dict(zip(data_set['Action'].unique(),range(len(data_set['Action'].unique()))))
action_mapped

{'activate': 4,
 'buy': 3,
 'cancel': 2,
 'change': 7,
 'enquire': 0,
 'garbage': 6,
 'report': 1,
 'request': 5}

In [35]:
object_mapped = dict(zip(data_set['Object'].unique(),range(len(data_set['Object'].unique()))))
object_mapped

{'balance': 7,
 'balance_minutes': 18,
 'bill': 9,
 'contact': 24,
 'credit': 10,
 'detail': 8,
 'garbage': 20,
 'idd': 19,
 'information': 16,
 'internet': 3,
 'iservice': 13,
 'lost_stolen': 17,
 'loyalty_card': 23,
 'mobile_setting': 12,
 'nontruemove': 6,
 'officer': 25,
 'package': 1,
 'payment': 0,
 'phone_issues': 4,
 'promotion': 11,
 'rate': 22,
 'ringtone': 21,
 'roaming': 14,
 'service': 5,
 'suspend': 2,
 'truemoney': 15}

In [36]:
data_set['Action'] = data_set['Action'].apply(lambda r : action_mapped[r])
data_set['Action'][:3]

0    0
1    0
2    1
Name: Action, dtype: int64

In [37]:
data_set['Object'] = data_set['Object'].apply(lambda r : object_mapped[r])
data_set['Object'][:3]

0    0
1    1
2    2
Name: Object, dtype: int64

In [38]:
X_train = sequence.pad_sequences(dataset, maxlen=max_len, padding='post', truncating='pre') #padding
y_action_train = data_set['Action'].as_matrix()
y_object_train = data_set['Object'].as_matrix()

y_action_test = list(y_action_train[int(len(y_action_train)*0.85):])
y_action_train = y_action_train[:int(len(y_action_train)*0.85)]

y_object_test = list(y_object_train[int(len(y_object_train)*0.85):])
y_object_train = y_object_train[:int(len(y_object_train)*0.85)]

y_action_train = pd.get_dummies(y_action_train).as_matrix()
y_object_train = pd.get_dummies(y_object_train).as_matrix()

X_test = X_train[int(len(X_train)*0.85):]
X_train = X_train[:int(len(X_train)*0.85)]

print(len(X_train), len(X_test), len(y_action_test),len(y_object_test))

11373 2007 2007 2007


In [39]:
match = 0
for w in dictionary.keys():
    if(w.strip('\n') in ftext_w.keys()):
        match += 1

print('match', match, match/len(dictionary))

match 917 0.5094444444444445


In [22]:
### Prepare embed layer
def prepare_embed(pretrain):
    pre_emb = []
    pre_emb.append(np.zeros(300))

    for k in dictionary.keys():
        if(not k in pretrain.keys()):    
            pre_emb.append(np.zeros(300))
        else:
            if(len(pretrain[k]) ==  300):
                pre_emb.append(pretrain[k])
            else:
                pre_emb.append(np.zeros(300))
    return pre_emb

pre_emb = np.array(prepare_embed(ftext_w))

In [23]:
from sklearn.metrics import f1_score,precision_score,recall_score
def evaluate(x_test, y_test, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)
    #map probability to class
    y_pred_mapped = []
    for i,pred in enumerate(y_pred):
        pred = list(pred)
        y_pred_mapped.append(pred.index(max(pred)))    
    
    f1score = f1_score(y_test,y_pred_mapped, average='weighted')
    precision = precision_score(y_test,y_pred_mapped, average='weighted')
    recall = recall_score(y_test,y_pred_mapped, average='weighted')
    return f1score, precision, recall

## #TODO 3: Build and evaluate a model for "action" classification


In [216]:
#TODO 3: Build and evaluate a model for "action" classification
## Predict Model
def get_predict_model():
    input1 = Input(shape=(max_len,))
    x = Embedding(len(dictionary)+1, 300, weights=[pre_emb], trainable=True)(input1)
    x = Conv1D(32,5,strides=1,activation='relu',padding="same")(x)
#     x = MaxPooling1D(pool_size=5, strides=1, padding='same')(x)    
    x = Dropout(0.2)(x)
    x = Conv1D(32,5,strides=1,activation='relu',padding="same")(x)
#     x = MaxPooling1D(pool_size=5, strides=1, padding='same')(x)   
    x = TimeDistributed(Dense(5))(x)
    x = Flatten()(x)
    x = Dense(64, activation='relu')(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.2)(x)
    out = Dense(8, activation='softmax')(x)
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])          
    return model
model = get_predict_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_33 (InputLayer)        (None, 55)                0         
_________________________________________________________________
embedding_32 (Embedding)     (None, 55, 300)           540300    
_________________________________________________________________
conv1d_42 (Conv1D)           (None, 55, 32)            48032     
_________________________________________________________________
dropout_66 (Dropout)         (None, 55, 32)            0         
_________________________________________________________________
conv1d_43 (Conv1D)           (None, 55, 32)            5152      
_________________________________________________________________
time_distributed_23 (TimeDis (None, 55, 5)             165       
_________________________________________________________________
flatten_16 (Flatten)         (None, 275)               0         
__________

In [217]:
%%time

weight_path_model_best='/data/truevoice-action.h5'

callbacks_list = [
#    TensorBoard(log_dir='/data/Graph/midterm', histogram_freq=1, write_grads=True),
    ModelCheckpoint(
        weight_path_model_best,
        monitor = "val_loss",
        mode = 'min',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                    patience=2, min_lr=0.001)
]

model.fit(X_train,y_action_train,batch_size=256,epochs=10,verbose=1, validation_split=0.2, shuffle=True, callbacks=callbacks_list)

Train on 9098 samples, validate on 2275 samples
Epoch 1/10
9098/9098 [==============================] - 4s - loss: 1.5134 - categorical_accuracy: 0.5672 - val_loss: 1.0490 - val_categorical_accuracy: 0.7257
Epoch 2/10
9098/9098 [==============================] - 0s - loss: 1.1958 - categorical_accuracy: 0.6101 - val_loss: 0.8943 - val_categorical_accuracy: 0.7257
Epoch 3/10
9098/9098 [==============================] - 0s - loss: 1.0446 - categorical_accuracy: 0.6452 - val_loss: 0.7945 - val_categorical_accuracy: 0.7613
Epoch 4/10
9098/9098 [==============================] - 0s - loss: 0.8754 - categorical_accuracy: 0.7017 - val_loss: 0.7237 - val_categorical_accuracy: 0.7789
Epoch 5/10
9098/9098 [==============================] - 0s - loss: 0.7345 - categorical_accuracy: 0.7588 - val_loss: 0.6550 - val_categorical_accuracy: 0.7943
Epoch 6/10
9098/9098 [==============================] - 0s - loss: 0.5980 - categorical_accuracy: 0.8073 - val_loss: 0.5668 - val_categorical_accuracy: 0.830

In [276]:
evaluate(X_test, y_action_test, model)

/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.8041230511277234, 0.786206807495494, 0.8256103637269556)

## #TODO 4: Build and evaluate a model for "object" classification



In [59]:
#TODO 4: Build and evaluate a model for "object" classification
## Predict Model
def get_object_model():
    input1 = Input(shape=(max_len,))
    x = Embedding(len(dictionary)+1, 300, weights=[pre_emb], trainable=True)(input1)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
    x = Dropout(0.2)(x)
    x = TimeDistributed(Dense(100))(x)
    x = Bidirectional(GRU(100))(x)
#     x = Flatten()(x)
    x = Dropout(0.2)(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)    
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.2)(x)
    
    out = Dense(26, activation='softmax')(x)
    
    model = Model(inputs=input1, outputs=out)
    model.compile(optimizer=Adam(),
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])          
    return model
model_2 = get_object_model()
model_2.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 55)                0         
_________________________________________________________________
embedding_6 (Embedding)      (None, 55, 300)           540300    
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 55, 100)           150100    
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 55, 100)           50100     
_________________________________________________________________
dropout_20 (Dropout)         (None, 55, 100)           0         
_________________________________________________________________
time_distributed_6 (TimeDist (None, 55, 100)           10100     
_________________________________________________________________
bidirectional_6 (Bidirection (None, 200)               120600    
__________

In [60]:
weight_path_model_obj='/data/truevoice-object.h5'

obj_callbacks_list = [
#    TensorBoard(log_dir='/data/Graph/midterm', histogram_freq=1, write_grads=True),
    ModelCheckpoint(
        weight_path_model_obj,
        monitor = "val_loss",
        mode = 'min',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                    patience=2, min_lr=0.001)
]

In [61]:
model_2.fit(X_train,y_object_train,batch_size=256,epochs=10,verbose=1, validation_split=0.15, shuffle=True, callbacks=obj_callbacks_list)

Train on 9667 samples, validate on 1706 samples
Epoch 1/10
9667/9667 [==============================] - 7s - loss: 2.8182 - categorical_accuracy: 0.1504 - val_loss: 2.5729 - val_categorical_accuracy: 0.1260
Epoch 2/10
9667/9667 [==============================] - 6s - loss: 2.4578 - categorical_accuracy: 0.2397 - val_loss: 2.1864 - val_categorical_accuracy: 0.3798
Epoch 3/10
9667/9667 [==============================] - 6s - loss: 1.9964 - categorical_accuracy: 0.4156 - val_loss: 1.7323 - val_categorical_accuracy: 0.4683
Epoch 4/10
9667/9667 [==============================] - 6s - loss: 1.6179 - categorical_accuracy: 0.5130 - val_loss: 1.5102 - val_categorical_accuracy: 0.5199
Epoch 5/10
9667/9667 [==============================] - 6s - loss: 1.3824 - categorical_accuracy: 0.5753 - val_loss: 1.4221 - val_categorical_accuracy: 0.5574
Epoch 6/10
9667/9667 [==============================] - 6s - loss: 1.1885 - categorical_accuracy: 0.6355 - val_loss: 1.3312 - val_categorical_accuracy: 0.593

In [62]:
model_2.load_weights(weight_path_model_obj)

In [63]:
evaluate(X_test, y_object_test, model_2)

/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


(0.6188333780285895, 0.626024551613759, 0.6347782760338814)

## #TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go 

This can be a bit tricky, if you are not familiar with the Keras functional API. PLEASE READ this webpage(https://keras.io/getting-started/functional-api-guide/) before you start this task.   

Your model will have 2 separate output layers one for action classification task and another for object classification task. 

This is a rough sketch of what your model might look like:
![image](https://raw.githubusercontent.com/ekapolc/nlp_course/master/HW5/multitask_sketch.png)

In [145]:
#TODO 5: Build and evaluate a multi-task model that does both "action" and "object" classifications in one-go
## Predict Model
def get_predict_model_3():
    input1 = Input(shape=(max_len,))
    x = Embedding(len(dictionary)+1, 300, weights=[pre_emb], trainable=True)(input1)
#     x = Conv1D(100,5,strides=1,activation='relu',padding="same")(x)
#     x = MaxPooling1D(pool_size=5, strides=1, padding='same')(x)   
#     x = TimeDistributed(Dense(100))(x)
#     x = Dropout(0.25)(x)
    x = Bidirectional(LSTM(100,return_sequences=True))(x)    
    x = Dropout(0.25)(x)
    x = Bidirectional(LSTM(100))(x)
#     x = Flatten()(x)
    x = Dropout(0.25)(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dense(100, activation='relu')(x)
    x = Dropout(0.25)(x)
    out1 = Dense(8, activation='softmax')(x)
    out2 = Dense(26, activation='softmax')(x)
    model = Model(inputs=input1, outputs=[out1,out2])
    model.compile(optimizer=Adam(),
                 loss='categorical_crossentropy',
                 metrics=['categorical_accuracy'])          
    return model
model_3 = get_predict_model_3()
model_3.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_30 (InputLayer)            (None, 55)            0                                            
____________________________________________________________________________________________________
embedding_30 (Embedding)         (None, 55, 300)       540300      input_30[0][0]                   
____________________________________________________________________________________________________
bidirectional_35 (Bidirectional) (None, 55, 200)       320800      embedding_30[0][0]               
____________________________________________________________________________________________________
dropout_74 (Dropout)             (None, 55, 200)       0           bidirectional_35[0][0]           
___________________________________________________________________________________________

In [146]:
weight_path_model_obj='/data/truevoice-both.h5'

both_callbacks_list = [
#    TensorBoard(log_dir='/data/Graph/midterm', histogram_freq=1, write_grads=True),
    ModelCheckpoint(
        weight_path_model_obj,
        monitor = "val_loss",
        mode = 'min',
        verbose = 1,
        save_best_only = True,
        save_weights_only = True,
    ),
    ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                    patience=2, min_lr=0.001)
]

In [147]:
model_3.fit(X_train,[y_action_train,y_object_train],batch_size=256,epochs=15,verbose=1, validation_split=0.2, shuffle=True, callbacks=both_callbacks_list)

Train on 9098 samples, validate on 2275 samples
Epoch 1/15
9098/9098 [==============================] - 17s - loss: 4.4712 - dense_128_loss: 1.5198 - dense_129_loss: 2.9513 - dense_128_categorical_accuracy: 0.5805 - dense_129_categorical_accuracy: 0.1418 - val_loss: 3.7401 - val_dense_128_loss: 1.0672 - val_dense_129_loss: 2.6729 - val_dense_128_categorical_accuracy: 0.7257 - val_dense_129_categorical_accuracy: 0.1174
Epoch 2/15
9098/9098 [==============================] - 13s - loss: 3.8946 - dense_128_loss: 1.2632 - dense_129_loss: 2.6313 - dense_128_categorical_accuracy: 0.6093 - dense_129_categorical_accuracy: 0.1783 - val_loss: 3.2816 - val_dense_128_loss: 0.9130 - val_dense_129_loss: 2.3685 - val_dense_128_categorical_accuracy: 0.7415 - val_dense_129_categorical_accuracy: 0.2985
Epoch 3/15
9098/9098 [==============================] - 13s - loss: 3.1894 - dense_128_loss: 0.9573 - dense_129_loss: 2.2321 - dense_128_categorical_accuracy: 0.6736 - dense_129_categorical_accuracy: 0.31

In [113]:
from sklearn.metrics import f1_score,precision_score,recall_score
def evaluate2(x_test, y_test, y_test2, model):
    """
    Evaluate model on the splitted 10 percent testing set.
    """
    y_pred = model.predict(x_test)
    
    #map probability to class
    y_pred_obj_mapped = []
    y_pred_act_mapped = []
    
    for pred_act in y_pred[0]:
        pred_act = list(pred_act)
        y_pred_act_mapped.append(pred_act.index(max(pred_act)))
    
    for pred_obj in y_pred[1]:
        pred_obj = list(pred_obj)
        y_pred_obj_mapped.append(pred_obj.index(max(pred_obj)))
    
    f1score = f1_score(y_test,y_pred_act_mapped, average='weighted')
    precision = precision_score(y_test,y_pred_act_mapped, average='weighted')
    recall = recall_score(y_test,y_pred_act_mapped, average='weighted')
    
    f1score2 = f1_score(y_test2,y_pred_obj_mapped, average='weighted')
    precision2 = precision_score(y_test2,y_pred_obj_mapped, average='weighted')
    recall2 = recall_score(y_test2,y_pred_obj_mapped, average='weighted')
    print("act {}, {}, {}\nobj {}, {}, {}\n".format(f1score,precision,recall,f1score2,precision2,recall2))
    acc = 0
    for i in range(len(x_test)):
        if( y_pred_act_mapped[i] == y_test[i] and y_pred_obj_mapped[i] == y_test2[i]):
            acc += 1
    print("acc: {}".format(acc/len(x_test)))
#     return f1score, precision, recall,

In [149]:
model_3.load_weights(weight_path_model_obj)

In [150]:
evaluate2(X_test, y_action_test, y_object_test, model_3)

act 0.8162753041497557, 0.7953877815318967, 0.8390632785251619
obj 0.5750871407405673, 0.6177382904499122, 0.5959142999501744

acc: 0.5705032386646737


/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/home/ekapolc/.env/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
